Workbook scraping data from Wikipedia into dataframe

Load all needed libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Get the page

In [2]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page = requests.get(wikipedia_link).text

In [3]:
soup = BeautifulSoup(wiki_page,'lxml')

Find table on page and all rows from table

In [4]:
toronto_pc_tab = soup.table    # postcodes table
pc_rows = toronto_pc_tab.find_all('tr')

In [5]:
postcodes=[]
boroughs=[]
neighborhoods=[]

Process all table rows, ignore Not Assigned Boroughs.
Move Borough name into Neighborhood if Not Assigned.

In [6]:
for row in pc_rows[1:]:
    
    postcode = str(row.find_all('td')[0].text)
    borough = str(row.find_all('td')[1].text)
    neighborhood = str(row.find_all('td')[2].text)
    neighborhood=neighborhood.strip('\n')
    if borough != 'Not assigned':
        postcodes.append(postcode)
        boroughs.append(borough)
        if neighborhood != 'Not assigned':
            neighborhoods.append(neighborhood)
        else:
            neighborhoods.append(borough)

Build data frame then group by Post Codes and Boroughs - join all Neighborhoods into one list.

In [8]:
cols = {'PostalCode':postcodes,'Borough':boroughs,'Neighborhood':neighborhoods}
df = pd.DataFrame(cols)

dfc = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

dfc.head()



,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [9]:
dfc.shape

(103, 3)